In [1]:
%autosave 60

import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import batch_norm, dropout

Autosaving every 60 seconds


Exercise 8
===

Create a DQN to solve the "BipedalWalker-v2" gym.

In [2]:
import gym

env = gym.make("BipedalWalker-v2")
obs = env.reset()
print(obs)

[ 2.74708518e-03  2.86767026e-06 -3.73954531e-04 -1.60000181e-02
  9.22417939e-02  8.68125702e-04  8.60060006e-01  7.13005662e-04
  1.00000000e+00  3.26231159e-02  8.68074829e-04  8.53662491e-01
 -5.90644195e-04  1.00000000e+00  4.40813750e-01  4.45819855e-01
  4.61422503e-01  4.89549905e-01  5.34102499e-01  6.02460682e-01
  7.09148467e-01  8.85931313e-01  1.00000000e+00  1.00000000e+00]


In [3]:
from matplotlib import pyplot as plt

%matplotlib inline

def render_env(env):
    plt.imshow(env.render(mode="rgb_array"))

#render_env(env)

In [4]:
print(env.action_space)
print(env.action_space.shape)
print(env.action_space.low)
print(env.action_space.high)

Box(4,)
(4,)
[-1 -1 -1 -1]
[1 1 1 1]


In [5]:
from tensorflow.contrib.layers import fully_connected

neurons_per_layer = [20]#[40, 40, 20]
learning_rate = 1 #0.001
n_discretized_actions = 3 # -1, -0.5, 0, +0.5, +1
n_actions = env.action_space.shape[0]
#n_outputs = env.action_space.shape[0]
n_outputs = n_discretized_actions ** n_actions
print("n_discretized_actions", n_discretized_actions, "n_actions", n_actions, "n_outputs", n_outputs)

def q_network(X_state, scope):
    current_layer = X_state
    layers = []
    with tf.variable_scope(scope) as scope:
        for neurons in neurons_per_layer:
            current_layer = fully_connected(current_layer, neurons)#, activation_fn=tf.nn.sigmoid)
            #print("current_layer shape", current_layer.get_shape())
        #print("input shape", current_layer.get_shape())
        outputs = fully_connected(current_layer, n_outputs, activation_fn=None)
    trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)
    trainable_vars_by_name = {var.name[len(scope.name)]: var for var in trainable_vars}
    return outputs, trainable_vars_by_name

n_discretized_actions 3 n_actions 4 n_outputs 81


In [6]:
print(obs.shape)

(24,)


In [7]:
X_state = tf.placeholder(tf.float32, shape=[None, obs.shape[0]])
actor_q_values, actor_vars = q_network(X_state, "q_networks/actor")
critic_q_values, critic_vars = q_network(X_state, "q_networks/critic")

copy_ops = [actor_var.assign(critic_vars[var_name])
            for var_name, actor_var in actor_vars.items()]
copy_critic_to_actor = tf.group(*copy_ops)

In [8]:
X_action = tf.placeholder(tf.int32, shape=[None, env.action_space.shape[0]])
# This contains the Q-value for the actor's chosen action.
# q_value = tf.reduce_sum(critic_q_values * tf.one_hot(X_action, n_outputs), axis=1, keep_dims=True)
q_value = tf.reduce_sum(critic_q_values * tf.one_hot(n_outputs, 1), axis=1, keep_dims=True)
#q_value = critic_q_values
print("critic_q_values.get_shape()", critic_q_values.shape, "q_value.get_shape()", q_value.get_shape())

critic_q_values.get_shape() (?, 81) q_value.get_shape() (?, 1)


In [9]:
# The Actor's reward + future discounted estimated Q-value.
y = tf.placeholder(tf.float32, shape=[None, 1])
cost = tf.reduce_mean(tf.square(y - q_value))
global_step = tf.Variable(0, trainable=False, name="global_step")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(cost, global_step=global_step)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
from collections import deque

replay_memory_size = 10000
replay_memory = deque([], maxlen=replay_memory_size)

def sample_memory(batch_size):
    indices = np.random.permutation(len(replay_memory))[:batch_size]
    cols = [[], [], [], [], []] # state, action, reward, next_state, continue
    for idx in indices:
        memory = replay_memory[idx]
        for col, value in zip(cols, memory):
            col.append(value)
    cols = [np.array(col) for col in cols]
    return (cols[0], cols[1], cols[2].reshape(-1, 1), cols[3], cols[4].reshape(-1, 1))


In [11]:
from itertools import product
discretized_actions = [-1, 0, 1]
action_space_universe = np.array(list(product(discretized_actions, discretized_actions, discretized_actions, discretized_actions)))

print(action_space_universe.shape)
#print(action_space_universe)

def action_index_to_action(action_index):
    return action_space_universe[action_index]

(81, 4)


In [ ]:
# epsilon greed algorithm to explore the state-action space of the environment at the beginning.
def epsilon_greedy(q_values, step):
    eps_min = 0.05 # At the start 5% of the time the actor will choose the greedy action.
    eps_max = 1.0 # Once it has finished exploring, the actor will choose the greedy action 100% of the time.
    eps_decay_steps = 200000
    
    epsilon = max(eps_min, eps_max - (eps_max - eps_min) * step / eps_decay_steps)
    if np.random.rand() < epsilon:
        #return np.random.uniform(env.action_space.low, env.action_space.high, size=n_actions)
        return np.random.randint(len(action_space_universe))
    else:
        #return q_values.reshape(n_outputs)
        return np.argmax(q_values) # the optimal/greedy action.

In [ ]:
import os

def preprocess_observation(obs):
    # I don't think I need this method.
    return obs

n_steps = 210000
# Start training the critic DQN after 1000 game iterations.
# This has to be a lot bigger than the batch_size defined below.
training_start = 1000
training_interval = 3 # Run a training step every 3 game iterations start training_start.
save_steps = 50
copy_steps = 25 # Copy the critic to the actor every 25 training steps.
discount_rate = 0.99999 # 0.95
batch_size = 50
iteration = 0
checkpoint_path = "./BipedalWalker-v2.ckpt"
done = True # Environment needs to be reset
total_reward = 0
current_steps = 0

with tf.Session() as sess:
    if os.path.isfile(checkpoint_path):
        print("Restoring checkpoint")
        saver.restore(sess, checkpoint_path)
    else:
        init.run()
    while True:
        step = global_step.eval()
        if step >= n_steps:
            break
        elif step != 0and step % 10000 == 0:
            print("global_step", step)
        iteration += 1
        if done:
            obs = env.reset()
            state = preprocess_observation(obs)
            total_reward = 0
            current_steps = 0
        
        # Actor evaluates what to do
        q_values = actor_q_values.eval(feed_dict={X_state: [state]})
        #print(q_values)
        action_index = epsilon_greedy(q_values, step)
        action = action_index_to_action(action_index)
        #print(action)
        
        #render_env(env)
        # Actor takes action
        obs, reward, done, info = env.step(action)
        if done:
            print("Agent died: current_steps", current_steps, "total_reward", total_reward)
            render_env(env)
        elif (150 < current_steps and current_steps % 10 == 0) or step > 90000:
            render_env(env)
        total_reward += reward
        current_steps += 1
        next_state = preprocess_observation(obs)
        
        # Memorise action.
        replay_memory.append((state, action, reward, next_state, 1.0 - done))
        state = next_state
        
        if iteration < training_start or iteration % training_interval != 0:
            continue
        
        # Train the critic.
        X_state_val, X_action_val, rewards, X_next_state_val, continues = (
            sample_memory(batch_size))
        next_q_values = actor_q_values.eval(feed_dict={X_state: X_next_state_val})
        max_next_q_values = np.max(next_q_values, axis=1, keepdims=True)
        # Calculate the Actor's reward + future discounted estimated Q-value.
        #print("rewards", rewards.shape, "continues", continues.shape, "max_next_q_values", max_next_q_values.shape)
        y_val = rewards + continues * discount_rate * max_next_q_values
        #print(X_state, X_action, y)
        #print("X_state_val", X_state_val.shape, "X_action", X_action_val.shape, "y_val", y_val.shape)
        training_op.run(feed_dict={X_state: X_state_val,
                                  X_action: X_action_val,
                                  y: y_val})
        
        if step % copy_steps == 0:
            copy_critic_to_actor.run()
        
        if step % save_steps == 0:
            saver.save(sess, checkpoint_path)

Agent died: current_steps 116 total_reward -23.803945232568903
Agent died: current_steps 113 total_reward -23.492575880540528
Agent died: current_steps 66 total_reward -15.643291910674412
Agent died: current_steps 90 total_reward -10.922980989407119
Agent died: current_steps 103 total_reward -19.14144876760615
Agent died: current_steps 76 total_reward -3.350739771795154
Agent died: current_steps 37 total_reward -8.322276695397372
Agent died: current_steps 45 total_reward -12.718892266169194
Agent died: current_steps 65 total_reward -15.53690377385231
Agent died: current_steps 63 total_reward -15.50765203927023
Agent died: current_steps 45 total_reward -7.822595762593051
Agent died: current_steps 1599 total_reward -110.34901866857302
Agent died: current_steps 1599 total_reward -126.32463917083741
Agent died: current_steps 78 total_reward -7.611501780005795
Agent died: current_steps 98 total_reward -17.195385325001553
Agent died: current_steps 55 total_reward -14.492614199799801
Agent di

Agent died: current_steps 75 total_reward -2.8807033831483424
Agent died: current_steps 74 total_reward -16.350544048570093
Agent died: current_steps 1599 total_reward -116.52148472407119
Agent died: current_steps 133 total_reward -15.960650776656944
Agent died: current_steps 100 total_reward -21.255030794510613
Agent died: current_steps 118 total_reward -3.5943775407063265
Agent died: current_steps 1599 total_reward -135.04876019959752
Agent died: current_steps 69 total_reward -4.711659635302298
Agent died: current_steps 1486 total_reward -133.30203322205944
Agent died: current_steps 70 total_reward -0.4402937989868227
Agent died: current_steps 96 total_reward -6.623934364743524
Agent died: current_steps 61 total_reward -17.34821325808763
Agent died: current_steps 99 total_reward -28.80870671799967
Agent died: current_steps 1599 total_reward -119.44251458073352
Agent died: current_steps 81 total_reward -1.3415354476794565
Agent died: current_steps 61 total_reward -18.348439663466067
A

Agent died: current_steps 129 total_reward -6.485736214259002
Agent died: current_steps 101 total_reward -4.77455969182029
Agent died: current_steps 149 total_reward -20.456552527287943
Agent died: current_steps 38 total_reward -4.183164329415189
Agent died: current_steps 77 total_reward -2.845057701524971
Agent died: current_steps 70 total_reward -19.688522764998172
Agent died: current_steps 103 total_reward -23.99079502249135
Agent died: current_steps 599 total_reward -71.98450719359084
Agent died: current_steps 63 total_reward -6.571508155960589
Agent died: current_steps 75 total_reward -2.611092600391559
Agent died: current_steps 78 total_reward -3.5031095197672677
Agent died: current_steps 62 total_reward -5.49128978076205
Agent died: current_steps 84 total_reward -0.13360821805956377
Agent died: current_steps 40 total_reward -11.062675286270679
Agent died: current_steps 1599 total_reward -114.07260619094752
Agent died: current_steps 44 total_reward -9.217416189596678
Agent died: 

Agent died: current_steps 105 total_reward -11.398429974893098
Agent died: current_steps 45 total_reward -4.119123979094127
Agent died: current_steps 47 total_reward -11.728435773423564
Agent died: current_steps 302 total_reward -31.495016551648504
Agent died: current_steps 57 total_reward -7.693122034346688
Agent died: current_steps 68 total_reward -19.5083536234113
Agent died: current_steps 81 total_reward -7.918883517909178
Agent died: current_steps 64 total_reward -17.763916383607928
Agent died: current_steps 76 total_reward -1.672378878562405
Agent died: current_steps 60 total_reward -2.669707521930961
Agent died: current_steps 67 total_reward -7.970459649636718
Agent died: current_steps 88 total_reward -23.29064004951094
Agent died: current_steps 1599 total_reward -109.95860336804232
Agent died: current_steps 1599 total_reward -110.76446184033249
Agent died: current_steps 1599 total_reward -131.84874814641378
Agent died: current_steps 163 total_reward -28.273433755559203
Agent di

Agent died: current_steps 92 total_reward -0.5173514868362464
Agent died: current_steps 1599 total_reward -128.34174589222903
Agent died: current_steps 59 total_reward -5.011976726893338
Agent died: current_steps 68 total_reward -15.733899857864406
Agent died: current_steps 56 total_reward -2.783477669229734
Agent died: current_steps 1599 total_reward -119.9460639490416
Agent died: current_steps 100 total_reward -23.53707064264516
Agent died: current_steps 50 total_reward -5.351259150027607
Agent died: current_steps 57 total_reward -3.2782764983984367
Agent died: current_steps 47 total_reward -3.7931750111151508
Agent died: current_steps 34 total_reward -12.689976281870159
Agent died: current_steps 94 total_reward -9.423848137706509
Agent died: current_steps 1599 total_reward -115.69352260065662
Agent died: current_steps 1599 total_reward -105.02359652569214
Agent died: current_steps 1599 total_reward -109.37818228451277
Agent died: current_steps 1383 total_reward -101.01731727027031
A

In [ ]:
print("Done")

For:
neurons_per_layer = [40, 10]
learning_rate = 0.001

The RL agent learnt how to get an average reward of 3 +- 2. I assume it is learning how to fall over. This is good progress. Running the agent with these hyper paremeters again, did not produce these same results unfortunately.

Rendering the environment has shown that a reward of 3-5 is probably just the agent falling as far forward as possible. Dumb robot.

After rendering the environment every 50 frames, I can see that it either falls over quickly or it splits it legs and very slowly inches forward and then gets killed after 1599 steps.

I couldn't get the agent to learn how to walk and when I checked my code, I realised that I was using the Q-values from the agent as the torque values for it's action, which basically made it stateless (I think).

Watching the agent, I could see that it was using too much force and flipping itself when it wanted to take a step. Let's add some granularity to the torque to see how it goes.